# Нейронные сети. Архитектуры нейронных сетей

## Подготовка данных

В этом практическом заданий мы будем решать задачу классификации цифр на датасете `mnist` с помощью полносвязной и сверточной нейронной сети. Для этого мы будем использовать надстройку над `tensorflow`, которая называется `keras`. Для начала обсудим данные. `mnist` датасет состоит из черно-белых изображений цифр размером $28 \times 28$ пикселей. В данном случае, мы работаем с одним каналом, хотя в случае цветных изображений, общее число каналов равно трем. Загрузим наши данные используя функцию `load_data` объекта `mnist` из модуля `keras.dataset`. Перед выполнением этого задания убедитесь, что ваша версия `tensorflow` >= 1.4.

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Нормализуйте заруженные данные `x_train` и `x_test`. Для этого следует разделить числовое значение каждого пикселя на $255$. Далее, переведите `y_train` и `y_test` в one-hot представление, используя функцию `tf.keras.utils.to_categorical`. Наше первое задание будет заключатся в реализации полносвязной нейронной сети. Поэтому измените размерность тренировочных и тестовых данных с помощью метода `reshape`.
    >> np_vector.shape
    >> (28, 28)
    >> np_vector = np_vector.reshape(28 * 28)
    >> np_vector.shape
    >> (784,)

### *РЕШЕНИЕ*

In [2]:
num_classes = 10 
x_train = x_train / 255
x_test = x_test/255 
y_train = tf.keras.utils.to_categorical(y_train,num_classes)
y_test = tf.keras.utils.to_categorical(y_test,num_classes)


In [18]:
x_train = x_train.reshape(x_train.shape[0],28*28)
x_test = x_test.reshape(x_test.shape[0],28*28)
x_train.shape

(60000, 784)

## Полносвязная нейронная сеть.

В этой части задания вам предлагается реализовать обычную нейронную сеть с использованием последовательной модели `tf.keras.models.Sequential`. Данная модель позволяет добавлять слои при помощи функции встроенной `add`. Наша нейронная сеть будет состоять всего лишь из одного скрытого слоя с количеством нейроннов равным $256$, функцией активации ReLU и с `input_shape=(784,)`, что соответствует количеству нейронов во входном слое нашей нейронной сети. Количество нейроннов в выходном слое равно количеству классов, в качестве функции активации нужно использовать softmax. Не забудьте вызвать `model.compile` после добавления слоев. Используйте в качестве функции потерь `categorical_crossentropy`, оптимизатор `adadelta` и метрику `accuracy`.

### *РЕШЕНИЕ*

In [21]:
batch_size = 128
epochs = 2
num_classes = 10 

In [26]:
import tensorflow as tf
from tensorflow.compat.v1.keras.layers import ReLU
from tensorflow.compat.v1.keras.layers import Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Dense(256,activation = 'relu' , input_shape=(784,)))
model.add(Dense(10,activation = 'softmax' , input_shape=(784,)))
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adadelta(),
    metrics = ['accuracy']
)
_ = model.fit(
    x_train,y_train,
#     batch_size = batch_size,
#     verbose = 1,
#     epochs = epochs,
#     validation_data = (x_test,y_test)
)


Train on 60000 samples
60000/60000 [==============================] - 16s 268us/sample - loss: 2.2268 - accuracy: 0.1911


In [28]:
loss, accuracy = model.evaluate(x_test,y_test,verbose = 0)
print(f'loss: {loss}')
print(f'accuract: {accuracy}')

loss: 2.0832107313156127
accuract: 0.33880001306533813


После этого, создайте модель и загрузите веса нейронной сети из файла `neural_networks.h5`. Какое количество настраиваемых параметров содержится в этой нейронной сети. Запишите это число в качестве первого ответа `answer1` на это задание. Оцените качетво на тестовой выборке и запишите это значение с точностью до трех знаков после запятой в переменную `answer2`.

In [31]:
model.load_weights('./neural_networks.h5')
loss, accuracy = model.evaluate(x_test,y_test,verbose = 0)
print(f'loss: {loss}')
print(f'accuract: {accuracy}')

loss: 0.0661844039292977
accuract: 0.9819999933242798


In [32]:
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_15 (Dense)             (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
2570 + 200960

203530

### *РЕШЕНИЕ*

## Сверточная нейронная сеть

Далее, вам предлагается реализовать сверточную нейронную сеть. 

* Размерность входного слоя $(28, 28, 1)$.
* Сверточный слой с $32$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Сверточный слой с $64$ каналами, ядро свертки $3 \times 3$.
* Макспулинг слой $(2,2)$.
* Понижение размерности признаков.
* Полносвязный слой с 64 нейронами
* Выходной слой с количеством нейронов равному количеству классов.

Для этого предлагается использовать следующие классы `Convolution2D`, `MaxPooling2D` и `Flatten` для понижения размерности признаков. Все эти классы как и слой полносвязной нейронной сети `Dense` находятся в `tf.keras.layers`. Используйте ReLU в качестве функции активации во всех слоях, где это потребуется, кроме выходного, в нем по аналогии с прошлым заданием используется softmax. Аналогичная ситуация с функцией `compile` после добавления слоев.

### *РЕШЕНИЕ*

Теперь оцените качество получившейся модели на тестовой выборке. Для этого измените размерность `x_train` и `x_test` на размерность входного слоя. Загрузите веса `conv_net.h5`. Запишите количество параметров этой сверточной нейронной сети в `answer3`. Сравните его с количеством параметром в полносвязной нейронной сети, которую мы реализовали ранее. Оценку качества запишите в `answer4` с точностью до 3 трех знаков после запятой.

### *РЕШЕНИЕ*

# Строка с ответами

In [ ]:
output = "nn params {0}\n nn score {1:.3f}\ncnn params {2}\ncnn score {3:.3f}"
print(output.format(answer1, answer2, answer3, answer4))